# FXa: Model Selection

In [1]:
import pandas as pd
import numpy as np
import glob, sys, os
sys.path.append('..')

In [2]:
from modules.plotting_metrics import PlotMetric
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white', context='talk', font_scale=0.8)

In [3]:
file_name = './df_DkSc_results_COCRYS_DEKOIS_DUD.pkl'
X_merged_dksc = pd.read_pickle(file_name)
# Extract activity column
y_true_merged = X_merged_dksc['activity']
# Drop column from merged_dkksc
X_merged_dksc = X_merged_dksc.drop('activity', axis=1)
X_merged_dksc.shape
y_true_merged.loc['DEKOIS'].sum()

40

### Timeout Decorator

In [4]:
import signal
from functools import wraps

def timeout(n_seconds=300):
    '''Stops a function execution after n seconds'''
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Set alarm for n seconds
            signal.alarm(n_seconds)
            try:
                # Call decorated func
                return func(*args, **kwargs)
#             except TimeoutError as e:
                print(f'Execution finished after {n_seconds}:', e)
            finally:
                # Cancel Alarm
                signal.alarm(0)
        return wrapper
    return decorator

## Scaffold Splitting

In [5]:
#*************************************************
# Functions to compute stratify scaffold splitting
#*************************************************
from scaffold_splitter import train_test_scaffold_split

In [6]:
# Compute or load the dataframe containing the Generic Murcko Scaffolds
file = './df_COCRYS_DUD_DEKOIS_Murcko_Scaffolds_SMILES.obj'

df_scff_murcko = pd.read_pickle(file)

## Train/test on the same dataset 

### Learning Curves

In [96]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve

In [8]:
def plot_learning_curves(estimator, X, y, title, ylim=[0.5,1], axes=None,
                         cv=3, train_sizes=np.linspace(0.1, 1.0, 10), 
                         scoring='roc_auc', n_jobs=4):
    '''
    Plot estimator performance on the training and validation
    sets as a function of the training set size.

    Parameters
    ----------
    estimator: sklearn estimator object type
       Object type that implements the "fit" and "predict method"

    X: array-like, shape (m_samples, n_features)
        Training array with m_samples and n_features.
    y: array-like, shape (m_samples)
        Target array relative to X with m labels.
    axes: array of 3 axes
        matplotlib axes array to append the generated plot
    ylim: array
       
    '''
    if axes == None:
        _, axes = plt.subplots(1, 1, figsize=(5, 5))
        
    axes.set(title=title, ylim=ylim, xlabel='Training examples', ylabel=f'Metric: {scoring}')
    # Use learning_curve function from sklearn
    train_sizes, train_scores, test_scores, fit_times, _ = \
       learning_curve(estimator, X, y, scoring=scoring, return_times = True,
                      cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    # Compute useful metrics
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
#     fit_times_mean = np.mean(fit_times, axis=1)
#     fit_times_std = np.std(fit_times, axis=1)
    # Plot the learning curve
    axes.grid()
    axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                        train_scores_mean + train_scores_std, alpha=0.1, color='r')
    axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                        test_scores_mean + test_scores_std, alpha=0.1, color='g')
    axes.plot(train_sizes, train_scores_mean, 'o-', color='r',
                label='Train score')
    axes.plot(train_sizes, test_scores_mean, 'o-', color='g',
                label='Cross-validation score')
    axes.legend(loc='lower right')
    

### Hyperparameters tunning: Grid Search

In [93]:
def run_grid_search(estimator, X_train, y_train, X_test, y_test, hyperparams,  cv_value=5, 
                    scoring='roc_auc', splitting='random', std_scale=False,
                   randomGS = True, n_iter=10):
    # Format the hyperparms
    hyperparams_dict = {'estimator__' + key: val for key, val in hyperparams.items()}
    
    if std_scale:
        # Create the Pipe
        scaler = StandardScaler()
        pipe = Pipeline([('scaler', scaler),
                         ('estimator', estimator)])
    else:
        pipe = estimator
    
    # Do Grid Search
    if randomGS:
        gs = RandomizedSearchCV(estimator = pipe,  param_distributions = hyperparams,
                     cv = cv_value, scoring = scoring, n_jobs = 6, refit = True, n_iter=n_iter)
    else:
        gs = GridSearchCV(estimator = pipe, param_grid = hyperparams,
                     cv = cv_value, scoring = scoring, n_jobs = 6, refit = Tirue)
    # Train the model
    gs.fit(X_train, y_train) 

    # Predictions
    y_train_predict = gs.predict_proba(X_train)
    y_test_predict = gs.predict_proba(X_test)
    
    # Values to return
    n_train_mols    = y_train.shape[0]
    n_train_actives = y_train.sum()
    n_test_mols     = y_test.shape[0]
    n_test_actives  = y_test.sum()
    mean_cv_roc     = gs.best_score_
    train_roc       = roc_auc_score(y_train, y_train_predict[:, 1])
    test_roc        = roc_auc_score(y_test, y_test_predict[:, 1])
    best_params     = gs.best_params_

    # Print some values
    print(f'No. of molecules in train set: {n_train_mols}, with {n_train_actives} actives.')
    print(f'No. of molecules in test set: {n_test_mols}, with {n_test_actives} actives.')
    print('')
    print('*'*10, 'GRID SEARCH RESULTS', '*'*10)
    print('- Mean CV ROC-AUC:\t{:.3f}'.format(mean_cv_roc))
    print('- Train ROC-AUC:  \t{:.3f}'.format(train_roc))
    print('- Test ROC-AUC:   \t{:.3f}'.format(test_roc))
    print('- Best hyperparameters', best_params)
    print('**'*21)
    print('')
    
    return [n_train_mols, n_train_actives, n_test_mols, n_test_actives,
            mean_cv_roc, train_roc, test_roc, best_params]

### Function to report the Best Conformation's ROC-AUC for a given subset of samples

In [10]:
#************************************************************************
# Returns the best conformatio's ROC-AUC value of a given subset X and y
#************************************************************************

def get_roc_auc_DkSc(X_train, y_train, X_test, y_test, verbose=True):
    roc_auc_train = X_train.apply(
        lambda x: roc_auc_score(y_true= y_train, y_score= -x), axis=0)
    roc_auc_test = X_test.apply(
        lambda x: roc_auc_score(y_true= y_test, y_score= -x), axis=0)
    # Values to return
    train_best_roc = roc_auc_train.max()
    train_median   = roc_auc_train.median()
    train_mean     = roc_auc_train.mean()
    test_best_roc = roc_auc_test.max()
    test_median   = roc_auc_test.median()
    test_mean     = roc_auc_test.mean()
    
    if verbose:
        print("***** Best Conformation's ROC-AUC using docking scores *****")
        
        print("> Train best conf. ROC-AUC: {:.3f}".format(train_best_roc) +
              " \t> median: {:.3f}".format(train_median) +
              ', mean: {:.3f}'.format(train_mean))
        
        print("> Test best conf. ROC-AUC: {:.3f}".format(test_best_roc) +
              " \t> median: {:.3f}".format(test_median) +
              ', mean: {:.3f}'.format(test_mean))
        print('**'*32)
    # return a list of results to capture by the  wrapper function
    return [train_best_roc, train_median, train_mean,
            test_best_roc, test_median, test_mean]
        

In [11]:
#******************************************
# Decorator functions to capture GS results
#******************************************
from functools import wraps

def capture_GS_results(results_dict=None, capture=True):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if results_dict != None and capture:
                results = func(*args, **kwargs)
                # Create a key with the first four values
                key = '_'.join(results[:4])
                # Append results to the results dictionary
                results_dict[key] = results
            else:
                return func(*args, **kwargs)
        return wrapper
    return decorator
            
#******************************************************
# Dictionary to capture GS results using the decorator 
#******************************************************
results_dict = {}

#******************************************************
# Function to Split and run Grid Search
#******************************************************
@capture_GS_results(results_dict)
def split_and_gs(train_name, test_name, estimator_name,
                 X, y, estimator, hyperparams, splitting='random', 
                 test_size=0.25, scaffold_series=None, **kwargs):
    '''Given a X and y sets, a sklean estimator and an splitting method, 
    performs Grid Search CV using the parsed hyperparams'''
    #**************
    # Do the split
    #**************
    if splitting == 'scaffold':
        X_train, X_test, y_train, y_test = \
            train_test_scaffold_split(X, y, scaffold_series = scaffold_series,
                test_size=test_size, stratify=y)
    elif splitting == 'random':
        X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=test_size, stratify=y)
        
    # Verbose
    print(f'{estimator_name} => Train: {train_name}; Test: {test_name}; split: {splitting}')
    
    # Function to run Grid Search
    #----------------------------
    gs_results = run_grid_search(estimator, 
                    X_train, y_train, X_test, y_test, 
                    hyperparams = hyperparams,  **kwargs)

    # Function to extract ROC results from DkSc values 
    #-------------------------------------------------
    dksc_results = get_roc_auc_DkSc(X_train, y_train, X_test, y_test)
    
    # Return both list of results to captured by the decorador function
    return [train_name, test_name, estimator_name, splitting] + gs_results + dksc_results

In [12]:
results_dict

{}

#  Hyperparameter Tunning: Grid Search
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with DEKOIS Library
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

In [13]:
#### library = 'DEKOIS'
library = 'DEKOIS'

# Train and test over DEKOIS
X = X_merged_dksc.loc[library]
y = y_true_merged.loc[library]
scaffold_series = df_scff_murcko['scff_generic'].loc[library]

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [14]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e4, 7)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LinearSVC => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.900
- Train ROC-AUC:  	0.943
- Test ROC-AUC:   	0.833
- Best hyperparameters {'C': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.910 	> median: 0.844, mean: 0.842
> Test best conf. ROC-AUC: 0.892 	> median: 0.801, mean: 0.799
****************************************************************
CPU times: user 1.05 s, sys: 447 ms, total: 1.49 s
Wall time: 6.24 s


In [15]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e4, 7)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
              X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LinearSVC => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.847
- Train ROC-AUC:  	0.875
- Test ROC-AUC:   	0.956
- Best hyperparameters {'C': 1e-08}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 636 ms, sys: 93.5 ms, total: 729 ms
Wall time: 1.2 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [16]:
%%time
estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e4, 7), 
               'gamma': np.geomspace(1e-8, 1e4, 7)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

rbfSVC => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.879
- Train ROC-AUC:  	0.888
- Test ROC-AUC:   	0.914
- Best hyperparameters {'C': 1e-08, 'gamma': 0.0001}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.887 	> median: 0.830, mean: 0.828
> Test best conf. ROC-AUC: 0.941 	> median: 0.853, mean: 0.841
****************************************************************
CPU times: user 1.57 s, sys: 407 ms, total: 1.98 s
Wall time: 11.5 s


In [17]:
%%time
estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e4, 7), 
               'gamma': np.geomspace(1e-8, 1e4, 7)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

rbfSVC => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.846
- Train ROC-AUC:  	0.875
- Test ROC-AUC:   	0.953
- Best hyperparameters {'C': 1e-08, 'gamma': 1e-06}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 1.69 s, sys: 313 ms, total: 2 s
Wall time: 10.7 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'> GS: Logistic Regression </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [18]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e4, 7), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LogReg => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.886
- Train ROC-AUC:  	0.912
- Test ROC-AUC:   	0.918
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.903 	> median: 0.834, mean: 0.830
> Test best conf. ROC-AUC: 0.944 	> median: 0.834, mean: 0.832
****************************************************************
CPU times: user 1.5 s, sys: 723 ms, total: 2.22 s
Wall time: 10 s


In [19]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e4, 7), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LogReg => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.863
- Train ROC-AUC:  	0.869
- Test ROC-AUC:   	0.953
- Best hyperparameters {'C': 1e-08, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 1.46 s, sys: 579 ms, total: 2.04 s
Wall time: 8.55 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: K-Neighbors Classifier</h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [20]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

kNN => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.883
- Train ROC-AUC:  	0.891
- Test ROC-AUC:   	0.849
- Best hyperparameters {'n_neighbors': 225, 'p': 1}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.909 	> median: 0.844, mean: 0.839
> Test best conf. ROC-AUC: 0.934 	> median: 0.807, mean: 0.807
****************************************************************
CPU times: user 896 ms, sys: 142 ms, total: 1.04 s
Wall time: 1.87 s


In [21]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

kNN => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.839
- Train ROC-AUC:  	0.871
- Test ROC-AUC:   	0.953
- Best hyperparameters {'n_neighbors': 225, 'p': 2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 826 ms, sys: 101 ms, total: 927 ms
Wall time: 1.31 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [22]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

DTree => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.816
- Train ROC-AUC:  	0.902
- Test ROC-AUC:   	0.807
- Best hyperparameters {'criterion': 'gini', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 0.3}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.896 	> median: 0.829, mean: 0.822
> Test best conf. ROC-AUC: 0.949 	> median: 0.861, mean: 0.857
****************************************************************
CPU times: user 1.43 s, sys: 363 ms, total: 1.79 s
Wall time: 2.51 s


In [23]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

DTree => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.851
- Train ROC-AUC:  	0.848
- Test ROC-AUC:   	0.759
- Best hyperparameters {'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 0.25}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 1.15 s, sys: 223 ms, total: 1.38 s
Wall time: 1.59 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Bagging Classifier (k-NN as base estimator) </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [24]:
%%time
from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# RANDOM Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 87.4 ms, sys: 62.4 ms, total: 150 ms
Wall time: 462 ms


In [25]:
%%time
# from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# SCAFFOLD Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 38 µs, sys: 9 µs, total: 47 µs
Wall time: 54.4 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [26]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

RandForest => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.871
- Train ROC-AUC:  	0.904
- Test ROC-AUC:   	0.938
- Best hyperparameters {'max_depth': 2, 'max_features': 'sqrt', 'min_samples_leaf': 0.2, 'n_estimators': 300}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.883 	> median: 0.821, mean: 0.816
> Test best conf. ROC-AUC: 0.960 	> median: 0.876, mean: 0.874
****************************************************************
CPU times: user 1.64 s, sys: 191 ms, total: 1.83 s
Wall time: 18.2 s


In [27]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

RandForest => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.856
- Train ROC-AUC:  	0.897
- Test ROC-AUC:   	0.945
- Best hyperparameters {'max_depth': 2, 'max_features': 'log2', 'min_samples_leaf': 0.2, 'n_estimators': 500}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 1.96 s, sys: 221 ms, total: 2.18 s
Wall time: 20.7 s


***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with DUD Library
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

In [28]:
library = 'DUD'

# Train and test over DUDU
X = X_merged_dksc.loc[library]
y = y_true_merged.loc[library]
scaffold_series = df_scff_murcko['scff_generic'].loc[library]

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [30]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LinearSVC => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 3669, with 106 actives.
No. of molecules in test set: 1224, with 35 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.834
- Train ROC-AUC:  	0.873
- Test ROC-AUC:   	0.826
- Best hyperparameters {'C': 0.001}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.733 	> median: 0.578, mean: 0.577
> Test best conf. ROC-AUC: 0.753 	> median: 0.552, mean: 0.551
****************************************************************
CPU times: user 1.92 s, sys: 122 ms, total: 2.04 s
Wall time: 9min 14s


In [31]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LinearSVC => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 3564, with 105 actives.
No. of molecules in test set: 1329, with 36 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.882
- Train ROC-AUC:  	0.911
- Test ROC-AUC:   	0.630
- Best hyperparameters {'C': 3.162277660168379e-06}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.747 	> median: 0.545, mean: 0.547
> Test best conf. ROC-AUC: 0.719 	> median: 0.640, mean: 0.638
****************************************************************
CPU times: user 1.63 s, sys: 140 ms, total: 1.77 s
Wall time: 4min 10s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [41]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

rbfSVC => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 3669, with 106 actives.
No. of molecules in test set: 1224, with 35 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.893
- Train ROC-AUC:  	0.988
- Test ROC-AUC:   	0.853
- Best hyperparameters {'C': 0.01, 'gamma': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.722 	> median: 0.579, mean: 0.578
> Test best conf. ROC-AUC: 0.729 	> median: 0.544, mean: 0.548
****************************************************************
CPU times: user 3.78 s, sys: 528 ms, total: 4.31 s
Wall time: 42.6 s


In [42]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

rbfSVC => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 3564, with 105 actives.
No. of molecules in test set: 1329, with 36 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.941
- Train ROC-AUC:  	0.997
- Test ROC-AUC:   	0.587
- Best hyperparameters {'C': 0.01, 'gamma': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.747 	> median: 0.545, mean: 0.547
> Test best conf. ROC-AUC: 0.719 	> median: 0.640, mean: 0.638
****************************************************************
CPU times: user 3.31 s, sys: 171 ms, total: 3.48 s
Wall time: 38.5 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'> GS: Logistic Regression </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [43]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LogReg => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 3669, with 106 actives.
No. of molecules in test set: 1224, with 35 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.860
- Train ROC-AUC:  	0.929
- Test ROC-AUC:   	0.858
- Best hyperparameters {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.724 	> median: 0.569, mean: 0.570
> Test best conf. ROC-AUC: 0.726 	> median: 0.570, mean: 0.574
****************************************************************
CPU times: user 5.24 s, sys: 508 ms, total: 5.74 s
Wall time: 41.3 s


In [44]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LogReg => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 3564, with 105 actives.
No. of molecules in test set: 1329, with 36 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.887
- Train ROC-AUC:  	0.967
- Test ROC-AUC:   	0.620
- Best hyperparameters {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.747 	> median: 0.545, mean: 0.547
> Test best conf. ROC-AUC: 0.719 	> median: 0.640, mean: 0.638
****************************************************************
CPU times: user 5.48 s, sys: 483 ms, total: 5.97 s
Wall time: 1min 3s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: K-Neighbors Classifier</h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [45]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

kNN => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 3669, with 106 actives.
No. of molecules in test set: 1224, with 35 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.811
- Train ROC-AUC:  	0.862
- Test ROC-AUC:   	0.877
- Best hyperparameters {'n_neighbors': 225, 'p': 1}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.739 	> median: 0.572, mean: 0.573
> Test best conf. ROC-AUC: 0.747 	> median: 0.560, mean: 0.563
****************************************************************
CPU times: user 5.9 s, sys: 180 ms, total: 6.08 s
Wall time: 22.3 s


In [46]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

kNN => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 3564, with 105 actives.
No. of molecules in test set: 1329, with 36 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.827
- Train ROC-AUC:  	0.951
- Test ROC-AUC:   	0.596
- Best hyperparameters {'n_neighbors': 55, 'p': 1}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.747 	> median: 0.545, mean: 0.547
> Test best conf. ROC-AUC: 0.719 	> median: 0.640, mean: 0.638
****************************************************************
CPU times: user 5.67 s, sys: 134 ms, total: 5.8 s
Wall time: 20.8 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [47]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

DTree => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 3669, with 106 actives.
No. of molecules in test set: 1224, with 35 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.771
- Train ROC-AUC:  	0.842
- Test ROC-AUC:   	0.813
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': None, 'min_samples_split': 0.25}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.727 	> median: 0.573, mean: 0.574
> Test best conf. ROC-AUC: 0.765 	> median: 0.561, mean: 0.562
****************************************************************
CPU times: user 1.62 s, sys: 202 ms, total: 1.82 s
Wall time: 3.38 s


In [48]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

DTree => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 3564, with 105 actives.
No. of molecules in test set: 1329, with 36 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.825
- Train ROC-AUC:  	0.875
- Test ROC-AUC:   	0.578
- Best hyperparameters {'criterion': 'gini', 'max_depth': 5, 'max_features': None, 'min_samples_split': 0.2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.747 	> median: 0.545, mean: 0.547
> Test best conf. ROC-AUC: 0.719 	> median: 0.640, mean: 0.638
****************************************************************
CPU times: user 1.71 s, sys: 142 ms, total: 1.86 s
Wall time: 3.33 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Bagging Classifier (k-NN as base estimator) </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [49]:
%%time
from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# RANDOM Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 233 µs, sys: 33 µs, total: 266 µs
Wall time: 279 µs


In [50]:
%%time
# from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# SCAFFOLD Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 43 µs, sys: 6 µs, total: 49 µs
Wall time: 57.9 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [51]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

RandForest => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 3669, with 106 actives.
No. of molecules in test set: 1224, with 35 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.777
- Train ROC-AUC:  	0.838
- Test ROC-AUC:   	0.810
- Best hyperparameters {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'n_estimators': 500}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.713 	> median: 0.568, mean: 0.568
> Test best conf. ROC-AUC: 0.754 	> median: 0.575, mean: 0.578
****************************************************************
CPU times: user 3.77 s, sys: 166 ms, total: 3.93 s
Wall time: 36.8 s


In [52]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

RandForest => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 3564, with 105 actives.
No. of molecules in test set: 1329, with 36 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.809
- Train ROC-AUC:  	0.888
- Test ROC-AUC:   	0.646
- Best hyperparameters {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'n_estimators': 500}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.747 	> median: 0.545, mean: 0.547
> Test best conf. ROC-AUC: 0.719 	> median: 0.640, mean: 0.638
****************************************************************
CPU times: user 3.27 s, sys: 105 ms, total: 3.38 s
Wall time: 32.7 s


***

<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with different libraries 
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

##### Here, Random or Scaffold Splitting are not used

In [53]:
# Train DEKOIS, test DUD
@capture_GS_results(results_dict)
def split_by_library_and_gs(train_name, test_name, estimator_name,
                            X, y, lib_train_name, lib_test_name, **kwargs):
    '''Train-Test "split" by library, and run Grid Search. Splits the main dataframe by library
    using different molecular libraries for Training and Testing.'''
    splitting='by_library'
    
    X_train = X.loc[lib_train_name]
    y_train = y.loc[lib_train_name]

    X_test = X.loc[lib_test_name]
    y_test = y.loc[lib_test_name]

    # Verbose
    print(f'{estimator_name} => Train: {train_name}; Test: {test_name}; split: {splitting}')
    
    # Function to run Grid Search
    #----------------------------
    gs_results = run_grid_search(estimator, 
                    X_train, y_train, X_test, y_test, 
                    hyperparams = hyperparams,  **kwargs)

    # Function to extract ROC results from DkSc values 
    #-------------------------------------------------
    dksc_results = get_roc_auc_DkSc(X_train, y_train, X_test, y_test)
    
    # Return both list of results to captured by the decorador function
    return [train_name, test_name, estimator_name, splitting] + gs_results + dksc_results

In [54]:
# **********************************************
# Train and Test sets from diferent libraries
# **********************************************
X = X_merged_dksc
y = y_true_merged

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM  </h3>
<b>Train and Test with different libraries</b>

In [55]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}
# Train DEKOIS, test DUD

train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

LinearSVC => Train: DEKOIS; Test: DUD; split: by_library
No. of molecules in train set: 1221, with 40 actives.
No. of molecules in test set: 4893, with 141 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.889
- Train ROC-AUC:  	0.894
- Test ROC-AUC:   	0.607
- Best hyperparameters {'C': 1e-08}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
> Test best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
****************************************************************
CPU times: user 705 ms, sys: 43.6 ms, total: 749 ms
Wall time: 1.75 s


In [56]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

LinearSVC => Train: DUD; Test: DEKOIS; split: by_library
No. of molecules in train set: 4893, with 141 actives.
No. of molecules in test set: 1221, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.821
- Train ROC-AUC:  	0.871
- Test ROC-AUC:   	0.716
- Best hyperparameters {'C': 0.0001}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
> Test best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
****************************************************************
CPU times: user 2.74 s, sys: 135 ms, total: 2.88 s
Wall time: 14min 44s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>Train and Test with different libraries</b>

In [57]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

estimator_name = 'rbfSVC'
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'gamma': np.geomspace(1e-8, 1e2, 6)}

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

rbfSVC => Train: DEKOIS; Test: DUD; split: by_library
No. of molecules in train set: 1221, with 40 actives.
No. of molecules in test set: 4893, with 141 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.891
- Train ROC-AUC:  	0.895
- Test ROC-AUC:   	0.604
- Best hyperparameters {'C': 1e-06, 'gamma': 1e-08}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
> Test best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
****************************************************************
CPU times: user 1.74 s, sys: 168 ms, total: 1.9 s
Wall time: 15.4 s


In [58]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

estimator_name = 'rbfSVC'
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'gamma': np.geomspace(1e-8, 1e2, 6)}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

rbfSVC => Train: DUD; Test: DEKOIS; split: by_library
No. of molecules in train set: 4893, with 141 actives.
No. of molecules in test set: 1221, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.869
- Train ROC-AUC:  	0.986
- Test ROC-AUC:   	0.658
- Best hyperparameters {'C': 1.0, 'gamma': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
> Test best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
****************************************************************
CPU times: user 8.89 s, sys: 475 ms, total: 9.36 s
Wall time: 6min 52s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Train and Test with different libraries</b>

In [66]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6),
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

LogReg => Train: DEKOIS; Test: DUD; split: by_library
No. of molecules in train set: 1221, with 40 actives.
No. of molecules in test set: 4893, with 141 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.891
- Train ROC-AUC:  	0.890
- Test ROC-AUC:   	0.603
- Best hyperparameters {'C': 0.0001, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
> Test best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
****************************************************************
CPU times: user 1.49 s, sys: 670 ms, total: 2.16 s
Wall time: 21 s


In [67]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

LogReg => Train: DUD; Test: DEKOIS; split: by_library
No. of molecules in train set: 4893, with 141 actives.
No. of molecules in test set: 1221, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.815
- Train ROC-AUC:  	0.922
- Test ROC-AUC:   	0.758
- Best hyperparameters {'C': 100.0, 'penalty': 'l2', 'solver': 'liblinear'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
> Test best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
****************************************************************
CPU times: user 2.19 s, sys: 532 ms, total: 2.72 s
Wall time: 49.5 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest  </h3>
<b>Train and Test with different libraries</b>

In [68]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [3,  5, 7],
             'min_samples_leaf': [0.03, 0.05],
               'min_samples_split': [0.2,  0.3],
             'max_features': ['sqrt']
            }

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

RandForest => Train: DEKOIS; Test: DUD; split: by_library
No. of molecules in train set: 1221, with 40 actives.
No. of molecules in test set: 4893, with 141 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.881
- Train ROC-AUC:  	0.929
- Test ROC-AUC:   	0.573
- Best hyperparameters {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 0.03, 'min_samples_split': 0.3, 'n_estimators': 300}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
> Test best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
****************************************************************
CPU times: user 2.64 s, sys: 106 ms, total: 2.75 s
Wall time: 44.1 s


In [69]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [3,  5, 7],
             'min_samples_leaf': [0.03, 0.05],
               'min_samples_split': [0.2,  0.3],
             'max_features': ['sqrt']
            }

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

RandForest => Train: DUD; Test: DEKOIS; split: by_library
No. of molecules in train set: 4893, with 141 actives.
No. of molecules in test set: 1221, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.802
- Train ROC-AUC:  	0.863
- Test ROC-AUC:   	0.675
- Best hyperparameters {'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 0.03, 'min_samples_split': 0.2, 'n_estimators': 300}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
> Test best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
****************************************************************
CPU times: user 4.36 s, sys: 147 ms, total: 4.5 s
Wall time: 1min 31s


***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Merged libraries 
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

#### DEKOIS, DUD and COCRYS are  treated as one unique library

In [94]:
# Train and test over 
X = X_merged_dksc
y = y_true_merged
library = 'Merged'
scaffold_series = df_scff_murcko['scff_generic']

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>Merged Libraries</b>

In [71]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LinearSVC => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.828
- Train ROC-AUC:  	0.870
- Test ROC-AUC:   	0.782
- Best hyperparameters {'C': 100.0}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.692 	> median: 0.614, mean: 0.612
> Test best conf. ROC-AUC: 0.668 	> median: 0.599, mean: 0.593
****************************************************************
CPU times: user 30min 38s, sys: 1.39 s, total: 30min 39s
Wall time: 1h 43s


In [72]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LinearSVC => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.809
- Train ROC-AUC:  	0.868
- Test ROC-AUC:   	0.724
- Best hyperparameters {'C': 1.0}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.686 	> median: 0.593, mean: 0.590
> Test best conf. ROC-AUC: 0.716 	> median: 0.650, mean: 0.645
****************************************************************
CPU times: user 14.6 s, sys: 481 ms, total: 15.1 s
Wall time: 26min 37s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM</h3>
<b>Merged Libraries</b>

In [73]:
%%time
from sklearn.svm import SVC

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C':np.geomspace(1e-8, 1e2, 6), 
               'gamma': np.geomspace(1e-8, 1e4, 7)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

rbfSVC => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.864
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.865
- Best hyperparameters {'C': 100.0, 'gamma': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.689 	> median: 0.621, mean: 0.618
> Test best conf. ROC-AUC: 0.689 	> median: 0.573, mean: 0.574
****************************************************************
CPU times: user 10.6 s, sys: 545 ms, total: 11.1 s
Wall time: 9min 12s


In [74]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'gamma': np.geomspace(1e-8, 1e2, 6)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

rbfSVC => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.853
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.774
- Best hyperparameters {'C': 100.0, 'gamma': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.686 	> median: 0.593, mean: 0.590
> Test best conf. ROC-AUC: 0.716 	> median: 0.650, mean: 0.645
****************************************************************
CPU times: user 9.19 s, sys: 341 ms, total: 9.53 s
Wall time: 6min 22s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Merged Libraries</b>

In [99]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LogReg => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.837
- Train ROC-AUC:  	0.896
- Test ROC-AUC:   	0.810
- Best hyperparameters {'solver': 'liblinear', 'penalty': 'l1', 'C': 1.0}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.703 	> median: 0.628, mean: 0.626
> Test best conf. ROC-AUC: 0.630 	> median: 0.550, mean: 0.549
****************************************************************
CPU times: user 6.71 s, sys: 431 ms, total: 7.14 s
Wall time: 20.6 s


In [76]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LogReg => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.817
- Train ROC-AUC:  	0.904
- Test ROC-AUC:   	0.749
- Best hyperparameters {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.686 	> median: 0.593, mean: 0.590
> Test best conf. ROC-AUC: 0.716 	> median: 0.650, mean: 0.645
****************************************************************
CPU times: user 7.16 s, sys: 456 ms, total: 7.62 s
Wall time: 45.7 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>Merged Libraries</b>

In [77]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2,  0.3],
               'min_samples_leaf': [0.02, 0.05, 0.1],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

DTree => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.731
- Train ROC-AUC:  	0.795
- Test ROC-AUC:   	0.731
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 0.05, 'min_samples_split': 0.2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.702 	> median: 0.620, mean: 0.618
> Test best conf. ROC-AUC: 0.682 	> median: 0.578, mean: 0.575
****************************************************************
CPU times: user 2.75 s, sys: 264 ms, total: 3.01 s
Wall time: 6.43 s


In [78]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2,  0.3],
               'min_samples_leaf': [0.02, 0.05, 0.1],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

DTree => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.727
- Train ROC-AUC:  	0.798
- Test ROC-AUC:   	0.653
- Best hyperparameters {'criterion': 'gini', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 0.05, 'min_samples_split': 0.3}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.686 	> median: 0.593, mean: 0.590
> Test best conf. ROC-AUC: 0.716 	> median: 0.650, mean: 0.645
****************************************************************
CPU times: user 2.71 s, sys: 276 ms, total: 2.99 s
Wall time: 6.36 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Bagging Classifier (k-NN as base estimator) </h3>
<b>Merged Libraries</b>

In [79]:
%%time
from sklearn.ensemble import BaggingClassifier

estimator_name = 'kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=3, oob_score=True)

hyperparams = {'n_estimators': [300]}

# RANDOM Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 236 µs, sys: 2 µs, total: 238 µs
Wall time: 246 µs


In [80]:
%%time
# from sklearn.ensemble import BaggingClassifier

estimator_name = 'kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=3, oob_score=True)

hyperparams = {'n_estimators': [300]}

# SCAFFOLD Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 50 µs, sys: 0 ns, total: 50 µs
Wall time: 58.7 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Random Forest </h3>
<b>Merged Libraries</b>

In [81]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 400],
               'max_depth': [2, 3],
               'min_samples_split': [0.1,  0.3],
               'min_samples_leaf': [0.02, 0.05],
               'max_features': ['sqrt']
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

RandForest => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.749
- Train ROC-AUC:  	0.791
- Test ROC-AUC:   	0.746
- Best hyperparameters {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 0.02, 'min_samples_split': 0.1, 'n_estimators': 300}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.678 	> median: 0.604, mean: 0.603
> Test best conf. ROC-AUC: 0.708 	> median: 0.622, mean: 0.619
****************************************************************
CPU times: user 2.98 s, sys: 76.4 ms, total: 3.06 s
Wall time: 37.2 s


In [82]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 400],
               'max_depth': [2, 3],
               'min_samples_split': [0.1,  0.3],
               'min_samples_leaf': [0.02, 0.05],
               'max_features': ['sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

RandForest => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.742
- Train ROC-AUC:  	0.799
- Test ROC-AUC:   	0.660
- Best hyperparameters {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 0.02, 'min_samples_split': 0.1, 'n_estimators': 300}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.686 	> median: 0.593, mean: 0.590
> Test best conf. ROC-AUC: 0.716 	> median: 0.650, mean: 0.645
****************************************************************
CPU times: user 3.01 s, sys: 212 ms, total: 3.22 s
Wall time: 37 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: X Gradient Boosting</h3>
<b>Merged Libraries</b>

In [97]:
%%time

from xgboost import XGBClassifier

estimator_name = 'XGB_tree'
estimator = XGBClassifier()
hyperparams = {'n_estimators': [200, 300],
               'max_depth': [2, 3, 10, 20],
               'learning_rate': [0.05, 0.1],
               'gamma': [0.01, 0.1, 0.5, 1],
               'alpha': [0.01, 0.1, 0.5, 1],
               'subsample': [0.3, 0.5],
               'colsample_bytree': [0.3, 0.5, 1]
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None,
             # RandomizedGS
            randomGS=True, n_iter=50)

XGB_tree => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.870
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.889
- Best hyperparameters {'subsample': 0.5, 'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 1, 'colsample_bytree': 0.5, 'alpha': 0.5}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.670 	> median: 0.602, mean: 0.599
> Test best conf. ROC-AUC: 0.728 	> median: 0.632, mean: 0.631
****************************************************************
CPU times: user 19 s, sys: 10.8 s, total: 29.7 s
Wall time: 3min 55s


In [98]:
%%time

from xgboost import XGBClassifier

estimator_name = 'XGB_tree'
estimator = XGBClassifier()
hyperparams = {'n_estimators': [200, 300],
               'max_depth': [2, 3, 10],
               'learning_rate': [0.05, 0.1],
               'gamma': [0.01, 0.1, 0.5, 1],
               'alpha': [0.01, 0.1, 0.5, 1],
               'subsample': [0.3, 0.5],
               'colsample_bytree': [0.3, 0.5, 1]
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series,
             # RandomizedGS
             randomGS=True, n_iter=50)

XGB_tree => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.834
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.774
- Best hyperparameters {'subsample': 0.5, 'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.05, 'gamma': 0.1, 'colsample_bytree': 0.3, 'alpha': 0.5}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.686 	> median: 0.593, mean: 0.590
> Test best conf. ROC-AUC: 0.716 	> median: 0.650, mean: 0.645
****************************************************************
CPU times: user 14.6 s, sys: 2.42 s, total: 17 s
Wall time: 3min 50s


## Get, Format, and Save the Results

In [101]:
### Save Results
row_names = ['Train_set', 'Test_set', 'Model name', 'Split', 
             'N_actives_train', 'N_actives_test', 'N_mols_train', 'Num_mols_test',
            'Mean-CV-ROC', 'ROC-AUC_train', 'ROC-AUC_test', 'best_params',
            'DkS_max_ROC_train',  'DkSc_med_ROC_train', 'DkSc_mean_ROC_train', 
            'DkS_max_ROC_test',  'DkSc_med_ROC_test', 'DkSc_mean_ROC_test']


# Load or save
save=False
filename='./GRID_SEARCH_CV_results.csv'
if save:
    results_df = pd.DataFrame(results_dict, index=row_names).T.reset_index().drop('index', axis=1)
    results_df.to_csv(filename)
else:
    results_df = pd.read_csv(filename)
    
results_df

,Train_set,Test_set,Model name,Split,N_actives_train,N_actives_test,N_mols_train,Num_mols_test,Mean-CV-ROC,ROC-AUC_train,ROC-AUC_test,best_params,DkS_max_ROC_train,DkSc_med_ROC_train,DkSc_mean_ROC_train,DkS_max_ROC_test,DkSc_med_ROC_test,DkSc_mean_ROC_test
0,DEKOIS,DEKOIS,LinearSVC,random,915,30,306,10,0.900377,0.942599,0.833446,{'C': 0.01},0.910471,0.843785,0.841723,0.891554,0.801436,0.798631
1,DEKOIS,DEKOIS,LinearSVC,scaffold,885,30,336,10,0.847173,0.874542,0.956442,{'C': 1e-08},0.883938,0.816355,0.81151,0.965491,0.896933,0.892571
2,DEKOIS,DEKOIS,rbfSVC,random,915,30,306,10,0.878908,0.887608,0.914189,"{'C': 1e-08, 'gamma': 0.0001}",0.886573,0.830169,0.827543,0.941047,0.853378,0.841224
3,DEKOIS,DEKOIS,rbfSVC,scaffold,885,30,336,10,0.846394,0.875127,0.952761,"{'C': 1e-08, 'gamma': 1e-06}",0.883938,0.816355,0.81151,0.965491,0.896933,0.892571
4,DEKOIS,DEKOIS,LogReg,random,915,30,306,10,0.886252,0.912354,0.918243,"{'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}",0.902881,0.834444,0.830249,0.943581,0.833868,0.832065
5,DEKOIS,DEKOIS,LogReg,scaffold,885,30,336,10,0.863158,0.868928,0.953374,"{'C': 1e-08, 'penalty': 'l2', 'solver': 'lbfgs'}",0.883938,0.816355,0.81151,0.965491,0.896933,0.892571
6,DEKOIS,DEKOIS,kNN,random,915,30,306,10,0.883145,0.890923,0.848986,"{'n_neighbors': 225, 'p': 1}",0.908644,0.843738,0.839167,0.933615,0.807264,0.807009
7,DEKOIS,DEKOIS,kNN,scaffold,885,30,336,10,0.839474,0.870643,0.953374,"{'n_neighbors': 225, 'p': 2}",0.883938,0.816355,0.81151,0.965491,0.896933,0.892571
8,DEKOIS,DEKOIS,DTree,random,915,30,306,10,0.815631,0.901883,0.807095,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",0.895518,0.829181,0.821594,0.948818,0.86098,0.85728
9,DEKOIS,DEKOIS,DTree,scaffold,885,30,336,10,0.85117,0.847797,0.759202,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",0.883938,0.816355,0.81151,0.965491,0.896933,0.892571


***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Merged libraries: Shuffle *y* target values in the train set
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

#### DEKOIS, DUD and COCRYS are  treated as one unique library

In [84]:
# Train and test over 
X = X_merged_dksc
# ***** Permutate y values *****
y = y_true_merged.sample(frac=1)

library = 'Merged'
scaffold_series = df_scff_murcko['scff_generic']


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>Merged Libraries</b>

In [85]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-9, 3)}

# RANDOM Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 300 µs, sys: 3 µs, total: 303 µs
Wall time: 262 µs


In [86]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-9, 3)}

# SCAFFOLD Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 310 µs, sys: 3 µs, total: 313 µs
Wall time: 270 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM</h3>
<b>Merged Libraries</b>

In [87]:
%%time
from sklearn.svm import SVC

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e0, 1e2, 3), 
               'gamma': np.geomspace(1e-4, 1e0, 3)}

# RANDOM Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 505 µs, sys: 5 µs, total: 510 µs
Wall time: 429 µs


In [88]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e0, 1e2, 3), 
               'gamma': np.geomspace(1e-4, 1e0, 3)}

# SCAFFOLD Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 540 µs, sys: 0 ns, total: 540 µs
Wall time: 446 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Merged Libraries</b>

In [89]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': np.geomspace(1e-4, 1e4, 3), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 329 µs, sys: 3 µs, total: 332 µs
Wall time: 293 µs


In [90]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': np.geomspace(1e-4, 1e5, 4), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 346 µs, sys: 3 µs, total: 349 µs
Wall time: 305 µs
